In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras   

In [ ]:
data=pd.read_csv("/content/IBM_2006-01-01_to_2018-01-01.csv")
data.head()

,Date,Open,High,Low,Close,Volume,Name
0,2006-01-03,82.45,82.55,80.81,82.06,11715200,IBM
1,2006-01-04,82.20,82.50,81.33,81.95,9840600,IBM
2,2006-01-05,81.40,82.90,81.00,82.50,7213500,IBM
3,2006-01-06,83.95,85.03,83.41,84.95,8197400,IBM
4,2006-01-09,84.10,84.25,83.38,83.73,6858200,IBM


In [ ]:
data.isnull().sum()

Date      0
Open      1
High      0
Low       1
Close     0
Volume    0
Name      0
dtype: int64

In [ ]:
data["Open"].fillna(data["Open"].median(),inplace=True)

In [ ]:
data = pd.read_csv("/content/IBM_2006-01-01_to_2018-01-01.csv", index_col='Date', parse_dates=["Date"])
data.head()

,Open,High,Low,Close,Volume,Name
Date,,,,,,
2006-01-03,82.45,82.55,80.81,82.06,11715200,IBM
2006-01-04,82.20,82.50,81.33,81.95,9840600,IBM
2006-01-05,81.40,82.90,81.00,82.50,7213500,IBM
2006-01-06,83.95,85.03,83.41,84.95,8197400,IBM
2006-01-09,84.10,84.25,83.38,83.73,6858200,IBM


In [ ]:
train = data[:'2016'].iloc[:,1:2].values
test = data['2017':].iloc[:,1:2].values

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
sc = MinMaxScaler(feature_range=(0,1))
train_scaled = sc.fit_transform(train)

In [ ]:
train_scaled

array([[0.06065089],
       [0.06029868],
       [0.06311637],
       ...,
       [0.66074951],
       [0.65546633],
       [0.6534235 ]])

In [ ]:
x_train = []
y_train = []

for i in range(60,2769):
    x_train.append(train_scaled[i-60:i, 0])
    y_train.append(train_scaled[i,0])

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
x_train.shape

(2709, 60, 1)

In [ ]:
model=keras.models.Sequential()
model.add(tf.keras.layers.SimpleRNN(50,return_sequences=True,input_shape=(x_train.shape[1],1),activation="tanh"))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.SimpleRNN(50,return_sequences=True,input_shape=(x_train.shape[1],1),activation="tanh"))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.GRU(50,return_sequences=True,input_shape=(x_train.shape[1],1),activation="tanh"))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.GRU(50,input_shape=(x_train.shape[1],1),activation="tanh"))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1))

In [ ]:
model.compile(optimizer='rmsprop', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
85/85 [==============================] - 14s 90ms/step - loss: 0.0853
Epoch 2/5
85/85 [==============================] - 8s 93ms/step - loss: 0.0174
Epoch 3/5
85/85 [==============================] - 8s 94ms/step - loss: 0.0106
Epoch 4/5
85/85 [==============================] - 8s 93ms/step - loss: 0.0074
Epoch 5/5
85/85 [==============================] - 8s 93ms/step - loss: 0.0083


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_18 (ModuleWra (None, 60, 50)            2600      
_________________________________________________________________
module_wrapper_19 (ModuleWra (None, 60, 50)            0         
_________________________________________________________________
module_wrapper_20 (ModuleWra (None, 60, 50)            5050      
_________________________________________________________________
module_wrapper_21 (ModuleWra (None, 60, 50)            0         
_________________________________________________________________
module_wrapper_22 (ModuleWra (None, 60, 50)            15300     
_________________________________________________________________
module_wrapper_23 (ModuleWra (None, 60, 50)            0         
_________________________________________________________________
module_wrapper_24 (ModuleWra (None, 50)               

In [ ]:
dataset_total = pd.concat((data['High'][:'2016'], data['High']['2017':]), axis=0)
print(dataset_total.shape)

inputs = dataset_total[len(dataset_total)-len(test)-60 : ].values
print(inputs.shape)
inputs = inputs.reshape(-1,1)
print(inputs.shape)
inputs = sc.transform(inputs)
print(inputs.shape)

(3020,)
(311,)
(311, 1)
(311, 1)


In [ ]:
x_test = []
for i in range(60,311):
    x_test.append(inputs[i-60:i, 0])
    
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
model.predict(x_test)

array([[0.6987963 ],
       [0.6963246 ],
       [0.69691366],
       [0.6988259 ],
       [0.70460147],
       [0.70850176],
       [0.71072525],
       [0.7095089 ],
       [0.7072788 ],
       [0.7047804 ],
       [0.70533663],
       [0.70677006],
       [0.70505947],
       [0.705007  ],
       [0.7055849 ],
       [0.7127328 ],
       [0.7248397 ],
       [0.7398503 ],
       [0.75360614],
       [0.7618493 ],
       [0.76434445],
       [0.76283664],
       [0.76147145],
       [0.75943846],
       [0.7605137 ],
       [0.76119256],
       [0.7613825 ],
       [0.762299  ],
       [0.7629021 ],
       [0.7667253 ],
       [0.7716534 ],
       [0.77700514],
       [0.7823228 ],
       [0.7876481 ],
       [0.79001105],
       [0.79206973],
       [0.79403645],
       [0.7951145 ],
       [0.7972393 ],
       [0.79552853],
       [0.79427147],
       [0.7933092 ],
       [0.79398066],
       [0.7941101 ],
       [0.7942979 ],
       [0.79428995],
       [0.7919701 ],
       [0.789